In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# upload file 
import pandas as pd
df= pd.read_excel('/content/drive/MyDrive/MC/Assessments/A2 & A3/scraped_data/Master files/df_SL.xlsx')

In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 56.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
df.drop(['ind'], axis=1, inplace=True)

In [ ]:
df.head()

,comment,stereotype
0,Another amazing video! So wonderful to see all...,0
1,"Beautiful people, love from Iran to my lovely ...",0
2,Love my nation Hazara,0
3,Be healthy friends,0
4,"Greetings, dear people",0


In [ ]:
df['stereotype'].value_counts()

0    295
1    254
Name: stereotype, dtype: int64

# Initial Run

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['comment'],df['stereotype'])

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
EVAL_METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer=Adam(learning_rate=0.00005),# LR
              loss='binary_crossentropy',
              metrics=EVAL_METRICS)

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

## Fit/ train the model

In [ ]:
model.fit(X_train, y_train, batch_size=16, epochs=15, validation_data=(X_test, y_test), callbacks=[early_stop]) # batch size and epochs

Epoch 1/15
26/26 [==============================] - 18s 91ms/step - loss: 0.9444 - accuracy: 0.5401 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.8862 - val_accuracy: 0.5290 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/15
26/26 [==============================] - 1s 43ms/step - loss: 0.8480 - accuracy: 0.5426 - precision: 1.0000 - recall: 0.0053 - val_loss: 0.7908 - val_accuracy: 0.5290 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/15
26/26 [==============================] - 1s 44ms/step - loss: 0.7659 - accuracy: 0.5401 - precision: 0.5000 - recall: 0.0212 - val_loss: 0.7363 - val_accuracy: 0.5290 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/15
26/26 [==============================] - 1s 44ms/step - loss: 0.7248 - accuracy: 0.5499 - precision: 0.5625 - recall: 0.0952 - val_loss: 0.7111 - val_accuracy: 0.5435 - val_precision: 0.7500 - val_recall: 0.0462
Epoch 5/15
26/26 [==============================] - 1s 44ms/step - loss: 0.

## Save

Save model then reload a fresh Keras model from the .keras zip archive

In [ ]:
# save the model 
model.save('/content/drive/MyDrive/MC/Assessments/A2 & A3/scraped_data/Master files/model2.keras')

# Evaluate

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

5/5 [==============================] - 1s 66ms/step


In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.53      0.85      0.66        73
           1       0.50      0.17      0.25        65

    accuracy                           0.53       138
   macro avg       0.52      0.51      0.45       138
weighted avg       0.52      0.53      0.47       138



# Tune 2 (run 2)

In [ ]:
EVAL_METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer=Adam(learning_rate=0.00005),# LR
              loss='binary_crossentropy',
              metrics=EVAL_METRICS)

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

## Fit/ train

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=15, validation_data=(X_test, y_test), callbacks=[early_stop]) # batch size and epochs

Epoch 1/15
13/13 [==============================] - 5s 181ms/step - loss: 0.6903 - accuracy: 0.5474 - precision: 0.5094 - recall: 0.4286 - val_loss: 0.6921 - val_accuracy: 0.5217 - val_precision: 0.4800 - val_recall: 0.1846
Epoch 2/15
13/13 [==============================] - 1s 97ms/step - loss: 0.6963 - accuracy: 0.5353 - precision: 0.4931 - recall: 0.3757 - val_loss: 0.6921 - val_accuracy: 0.5290 - val_precision: 0.5000 - val_recall: 0.1692
Epoch 3/15
13/13 [==============================] - 1s 95ms/step - loss: 0.6873 - accuracy: 0.5669 - precision: 0.5401 - recall: 0.3915 - val_loss: 0.6918 - val_accuracy: 0.5290 - val_precision: 0.5000 - val_recall: 0.1692
Epoch 4/15
13/13 [==============================] - 1s 95ms/step - loss: 0.6897 - accuracy: 0.5328 - precision: 0.4884 - recall: 0.3333 - val_loss: 0.6918 - val_accuracy: 0.5290 - val_precision: 0.5000 - val_recall: 0.1538
Epoch 5/15
13/13 [==============================] - 1s 94ms/step - loss: 0.6907 - accuracy: 0.5377 - precis

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

5/5 [==============================] - 1s 62ms/step


In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.54      0.85      0.66        73
           1       0.52      0.18      0.27        65

    accuracy                           0.54       138
   macro avg       0.53      0.52      0.47       138
weighted avg       0.53      0.54      0.48       138



# Tune 3 (run 3)

In [ ]:
EVAL_METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer=Adam(learning_rate=0.0001),# LR
              loss='binary_crossentropy',
              metrics=EVAL_METRICS)

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=15, validation_data=(X_test, y_test), callbacks=[early_stop]) # batch size and epochs

Epoch 1/15
13/13 [==============================] - 5s 193ms/step - loss: 0.7088 - accuracy: 0.5085 - precision: 0.4652 - recall: 0.4603 - val_loss: 0.6864 - val_accuracy: 0.5217 - val_precision: 0.4902 - val_recall: 0.3846
Epoch 2/15
13/13 [==============================] - 1s 95ms/step - loss: 0.6935 - accuracy: 0.5231 - precision: 0.4790 - recall: 0.4233 - val_loss: 0.6867 - val_accuracy: 0.5362 - val_precision: 0.5238 - val_recall: 0.1692
Epoch 3/15
13/13 [==============================] - 1s 96ms/step - loss: 0.6999 - accuracy: 0.5109 - precision: 0.4552 - recall: 0.3228 - val_loss: 0.6876 - val_accuracy: 0.5580 - val_precision: 0.6250 - val_recall: 0.1538
Epoch 4/15
13/13 [==============================] - 1s 96ms/step - loss: 0.6797 - accuracy: 0.5718 - precision: 0.5537 - recall: 0.3545 - val_loss: 0.6861 - val_accuracy: 0.5507 - val_precision: 0.5882 - val_recall: 0.1538
Epoch 5/15
13/13 [==============================] - 1s 98ms/step - loss: 0.6921 - accuracy: 0.5523 - precis

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

5/5 [==============================] - 1s 61ms/step


In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.59      0.77      0.67        73
           1       0.60      0.40      0.48        65

    accuracy                           0.59       138
   macro avg       0.60      0.58      0.57       138
weighted avg       0.60      0.59      0.58       138



# Tune 4 (run 4)

In [ ]:
EVAL_METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer=Adam(learning_rate=0.0002),# LR
              loss='binary_crossentropy',
              metrics=EVAL_METRICS)

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

## Fit/ train

In [ ]:
model.fit(X_train, y_train, batch_size=16, epochs=15, validation_data=(X_test, y_test), callbacks=[early_stop]) # batch size and epochs

Epoch 1/15
26/26 [==============================] - 6s 89ms/step - loss: 0.6954 - accuracy: 0.5401 - precision: 0.5000 - recall: 0.4444 - val_loss: 0.6764 - val_accuracy: 0.5870 - val_precision: 0.6250 - val_recall: 0.3077
Epoch 2/15
26/26 [==============================] - 1s 43ms/step - loss: 0.6781 - accuracy: 0.5523 - precision: 0.5185 - recall: 0.3704 - val_loss: 0.6759 - val_accuracy: 0.5725 - val_precision: 0.6250 - val_recall: 0.2308
Epoch 3/15
26/26 [==============================] - 1s 44ms/step - loss: 0.6766 - accuracy: 0.5815 - precision: 0.5491 - recall: 0.5026 - val_loss: 0.6732 - val_accuracy: 0.6159 - val_precision: 0.6500 - val_recall: 0.4000
Epoch 4/15
26/26 [==============================] - 1s 43ms/step - loss: 0.6805 - accuracy: 0.5742 - precision: 0.5660 - recall: 0.3175 - val_loss: 0.6719 - val_accuracy: 0.6232 - val_precision: 0.6667 - val_recall: 0.4000
Epoch 5/15
26/26 [==============================] - 1s 43ms/step - loss: 0.6899 - accuracy: 0.5547 - precisi

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

5/5 [==============================] - 1s 64ms/step


In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.61      0.86      0.71        73
           1       0.71      0.37      0.48        65

    accuracy                           0.63       138
   macro avg       0.66      0.62      0.60       138
weighted avg       0.65      0.63      0.60       138



# Tune 5 (run 5)

In [ ]:
EVAL_METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer=Adam(learning_rate=0.0004),# LR
              loss='binary_crossentropy',
              metrics=EVAL_METRICS)

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

In [ ]:
model.fit(X_train, y_train, batch_size=16, epochs=15, validation_data=(X_test, y_test), callbacks=[early_stop]) # batch size and epochs

Epoch 1/15
26/26 [==============================] - 5s 89ms/step - loss: 0.6657 - accuracy: 0.5669 - precision: 0.5359 - recall: 0.4339 - val_loss: 0.6543 - val_accuracy: 0.6449 - val_precision: 0.7000 - val_recall: 0.4308
Epoch 2/15
26/26 [==============================] - 1s 42ms/step - loss: 0.6593 - accuracy: 0.5961 - precision: 0.5878 - recall: 0.4074 - val_loss: 0.6556 - val_accuracy: 0.6304 - val_precision: 0.5795 - val_recall: 0.7846
Epoch 3/15
26/26 [==============================] - 1s 43ms/step - loss: 0.6731 - accuracy: 0.5718 - precision: 0.5436 - recall: 0.4286 - val_loss: 0.6549 - val_accuracy: 0.5870 - val_precision: 0.6667 - val_recall: 0.2462
Epoch 4/15
26/26 [==============================] - 1s 43ms/step - loss: 0.6586 - accuracy: 0.6034 - precision: 0.5684 - recall: 0.5714 - val_loss: 0.6484 - val_accuracy: 0.6304 - val_precision: 0.6944 - val_recall: 0.3846
Epoch 5/15
26/26 [==============================] - 1s 43ms/step - loss: 0.6485 - accuracy: 0.6107 - precisi

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

5/5 [==============================] - 1s 62ms/step


In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73        73
           1       0.70      0.66      0.68        65

    accuracy                           0.71       138
   macro avg       0.71      0.71      0.71       138
weighted avg       0.71      0.71      0.71       138



# Tune 6 (final run 6)

In [ ]:
EVAL_METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer=Adam(learning_rate=0.0008),# LR
              loss='binary_crossentropy',
              metrics=EVAL_METRICS)

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

In [ ]:
model.fit(X_train, y_train, batch_size=16, epochs=15, validation_data=(X_test, y_test), callbacks=[early_stop]) # batch size and epochs

Epoch 1/15
26/26 [==============================] - 5s 89ms/step - loss: 0.6290 - accuracy: 0.6691 - precision: 0.6710 - recall: 0.5503 - val_loss: 0.6272 - val_accuracy: 0.7391 - val_precision: 0.7164 - val_recall: 0.7385
Epoch 2/15
26/26 [==============================] - 1s 42ms/step - loss: 0.6462 - accuracy: 0.6034 - precision: 0.5878 - recall: 0.4603 - val_loss: 0.6319 - val_accuracy: 0.6812 - val_precision: 0.6296 - val_recall: 0.7846
Epoch 3/15
26/26 [==============================] - 1s 43ms/step - loss: 0.6290 - accuracy: 0.6496 - precision: 0.6573 - recall: 0.4974 - val_loss: 0.6406 - val_accuracy: 0.6159 - val_precision: 0.5625 - val_recall: 0.8308
Epoch 4/15
26/26 [==============================] - 1s 42ms/step - loss: 0.6432 - accuracy: 0.6277 - precision: 0.6023 - recall: 0.5608 - val_loss: 0.6184 - val_accuracy: 0.7029 - val_precision: 0.7069 - val_recall: 0.6308
Epoch 5/15
26/26 [==============================] - 1s 43ms/step - loss: 0.6331 - accuracy: 0.6302 - precisi

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

5/5 [==============================] - 1s 63ms/step


In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.74      0.71      0.73        73
           1       0.69      0.72      0.71        65

    accuracy                           0.72       138
   macro avg       0.72      0.72      0.72       138
weighted avg       0.72      0.72      0.72       138



Save a copy of this model 

In [ ]:
# save the model 
model.save('/content/drive/MyDrive/MC/Assessments/A2 & A3/scraped_data/Master files/model6.keras')

Try on unseen comments

# Try tuned model 6 on unseen comments

In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'Don\'t Be Scared; Get ANGRY! Fight Back! Weakness only emboldens Oppressors',  
    'Thank you so much France 24 and we appreciate your report from Hazara( Hazaristan).\
    The Hazara people are under the target of terrorist groups even the Afghan government doesn\'t help us and close our way toward modern life.\
We are making school, road, etc for ourselves but the Afghan government with the terrorist groups trying to stop us from modern life.\
Read more',
    'Deep sympathies to our Hazara bothers and sisters, but the King allegedly massacred and expelled, unjustly IMO, after a rebellion in Bamiyan.\
    An unfortunate history. Quetta was Afghanistan then, before Britain took it and made part of their Indian colony, which then became Pakistan. \
    Even now it is really Afghanistan culturally and ethnically. The presence of Pashtuns and Hazaras is a testimony. Read more',
    'It pains me so much to see persecution of Hazara community. You are in our prayers 🇮🇳',
    'I\'m a Sunni Muslim I don\'t hate anyone definitely on religious basses. I would even marry a Shia Muslimah.',
    'That old man looked like a badass.'
]

new_results = tf.sigmoid(model(tf.constant(examples)))
original_results = tf.sigmoid(model(tf.constant(examples)))

print('Results from the tuned model:')
print_my_examples(examples, new_results)

Results from the tuned model:
input: Don't Be Scared; Get ANGRY! Fight Back! Weakness only emboldens Oppressors : score: 0.638391
input: Thank you so much France 24 and we appreciate your report from Hazara( Hazaristan).    The Hazara people are under the target of terrorist groups even the Afghan government doesn't help us and close our way toward modern life.We are making school, road, etc for ourselves but the Afghan government with the terrorist groups trying to stop us from modern life.Read more : score: 0.612644
input: Deep sympathies to our Hazara bothers and sisters, but the King allegedly massacred and expelled, unjustly IMO, after a rebellion in Bamiyan.    An unfortunate history. Quetta was Afghanistan then, before Britain took it and made part of their Indian colony, which then became Pakistan.     Even now it is really Afghanistan culturally and ethnically. The presence of Pashtuns and Hazaras is a testimony. Read more : score: 0.603253
input: It pains me so much to see pe

In [ ]:
# reload model  (code for next file)
new_model = tf.keras.models.load_model(
       ('/content/drive/MyDrive/MC/Assessments/A2 & A3/scraped_data/Master files/model6.keras'),
       custom_objects={'KerasLayer':hub.KerasLayer})